# Webscraping von Stellenanzeigen – Korpuserstellung mit Hindernissen

**TU Dresden, Institut für Germanistik, Professur für Angewandte Linguistik**

***

**Seminar:** Programmieren lernen für Geistes- und Sozialwissenschaftler:innen

**Dozent:** Yannick Frommherz

**Semester der Prüfungsleistung:** Wintersemester 2023/2024

**Art der Prüfungsleistung:** Kombinierte Arbeit, Vertiefungsmodul Europäische Sprachen

***

**Eingereicht von:** Anna Bakker, Master EuroS, Matr.-Nr. 4068462

**Datum:** 31.03.2024

***

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}
r = requests.get("https://www.stepstone.de/jobs/in-Würzburg?radius=5&page=1", headers=headers, timeout=10)

In [ ]:
#str(r.status_code)

In [ ]:
import re
import os
from bs4 import BeautifulSoup
import requests 
import time

#‼️ headers als Argument übergeben, um Funktion autark zu machen
def search_urls(city, radius, page, headers):
    url = "https://www.stepstone.de/jobs/in-%s?radius=%i&page=%i" % (city, radius, page, headers) #abzurufende URL mit entsprechenden Platzhaltern
    print(url)
    #‼️ timeout einbauen
    response = requests.get(url, headers=headers, timeout=5) 
    
    ‼️ Handling fehlerhafter Status codes
    if not str(response.status_code).startswith("2"):
        return(response.status_code)
    
    quelltext = response.text
    
    regex = r'href="([^"]*\/stellenangebote-[^\s"]+)"' 
    url_matches = re.findall(regex, quelltext)
    print(url_matches)
    return url_matches

cities = ["Würzburg"] 
r = 5
til_page = 1 
#headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "de-DE,de;q=0.5","Accept-Encoding": "gzip, deflate"}
#‼️ Abfrage fuktioniert nur mit meinem header..., ggf. zwischen headern rotieren
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}
matches = [] 
for c in cities:
    print(c)
for p in range(1,til_page+1):
    print(p)
    found_urls = search_urls(c, r, p, headers)
    matches.extend(found_urls) 

for match in matches:
     print(match)

matches = list(set(matches))
print(len(matches))

In [ ]:
matches

In [ ]:
import requests
from bs4 import BeautifulSoup

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36'}

source_code = requests.get("https://www.mdr.de/nachrichten/sachsen-anhalt/landespolitik/hoecke-anklage-interview-kai-langer-gedenkstaetten-100.html", headers=headers, timeout=5)

In [ ]:
soup = BeautifulSoup(source_code.text)
soup.find("span", class_="headline").text

In [ ]:
#Definition der Funktion für die gewünschten Textblöcke
#‼️ nur Extraktion von .text anstatt kompletten Elementen, dann kann man sich auch die ganze Bereinigung sparen
def extract(soup_offer_text):
    title = ""
    introduction = ""
    description = ""
    profile = ""
    benefits = ""

    if (soup_offer_text.find(attrs={"data-at": "header-job-title"}) is None):
        return 
    else:
        title = str(soup_offer_text.find(attrs={"data-at": "header-job-title"}).contents[0]) 
    
    #‼️ ggf. Quelltext erst "zuschneiden"

#Greift auf den content innerhalb dieses Attributs zu; für jeden Anzeigenabschnitt einzeln
#Nimmt den Inhalt, wenn welcher da ist, aus den Attributen und dem span innerhalb der Attribute
    #‼️ if soup_offer_text.find(...):
    if (soup_offer_text.find(attrs={"data-at": "section-text-introduction-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-introduction-content"}).span is not None):
            introduction = str(soup_offer_text.find(attrs={"data-at": "section-text-introduction-content"}).span.contents[0]) 


    if (soup_offer_text.find(attrs={"data-at": "section-text-description-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-description-content"}).span is not None):
            description = str(soup_offer_text.find(attrs={"data-at": "section-text-description-content"}).span.contents[0])


    if (soup_offer_text.find(attrs={"data-at": "section-text-profile-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-profile-content"}).span is not None):
            profile = str(soup_offer_text.find(attrs={"data-at": "section-text-profile-content"}).span.contents[0])


    if (soup_offer_text.find(attrs={"data-at": "section-text-benefits-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-benefits-content"}).span is not None):
            benefits = str(soup_offer_text.find(attrs={"data-at": "section-text-benefits-content"}).span.contents[0])

    #‼️ xml oder csv für Strukturierung geeigneter
    texts = [introduction, description, profile, benefits]
    text = open("rohtext.txt", "a", encoding='utf-16-le')
    text.write("\n" + "## ")
    text.write("///" + title+ "///" + "\n")
    for x in texts:   
        text.write(x + "\n")
    text.close()

text = open("rohtext.txt", "w", encoding='utf-16-le')
text.close()
i = 0
for url in matches: #‼️ tqdm für Fortschrittsanzeige
    offer_url = "https://www.stepstone.de" + url #die extrahierten Linkbestandteile der Anzeigen werden durch den Stammlink komplettiert

    #headers (weil oben schon Yannicks Header) = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "de-DE,de;q=0.5","Accept-Encoding": "gzip, deflate"}

    sourcecode_offer = requests.get(offer_url, headers=headers).text
    soup_offer = BeautifulSoup(sourcecode_offer, "lxml")

    extract(soup_offer)
    i += 1
    
    print(str(i) + "/" + str(len(matches)) + "---" + format((i/len(matches))* 100, '.2f') + "%") 


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
stellenanzeigen = []
with open("Dachau.txt", encoding='utf-16-le') as f:
    dachau = f.read()
    for line in dachau.split("##"):
        stellenanzeigen.append(line.strip())

In [ ]:
df = pd.DataFrame([], columns=["Titel", "Text", "Form", "Ort", "Datum"])

In [ ]:
i = 0
for s in stellenanzeigen:
    if len(s) == 0:
        continue
    elemente = s.split("///")
    df.loc[i] = [elemente[1], elemente[2], "", "Dachau", "Ende März"]
    i+=1

In [ ]:
def tag(single_text):
    #hier müsste die Lemmatisierung hin 
    return single_text.upper()

In [ ]:
#Eintragen von Werten basierend auf Bedingungen
x = df.Titel
if_list = [x.str.contains('.\*(\s|\))'), x.str.contains('\w*(\*|:|_)\s*in')] 
then_list = ["Sternchen", "in-Form"]
df["Form"] = np.select(if_list, then_list, "None")

In [ ]:
df["Lemmata"] = df.Text.apply(tag)

In [ ]:
df.Form.value_counts()

In [ ]:
#Gruppieren und Verteilung berechnen
df.groupby("Ort").Form.value_counts(normalize=True)

In [ ]:
#Filtern
df[df.Form == "in-Form"].Titel

In [ ]:
import requests
from bs4 import BeautifulSoup 

#‼️ Funktion!
def extract(soup_offer_text):
    title = ""
    introduction = ""
    description = ""
    profile = ""
    benefits = ""

    if (soup_offer_text.find(attrs={"data-at": "header-job-title"}) is None):
        return 
    else:
        title = str(soup_offer_text.find(attrs={"data-at": "header-job-title"}).contents[0]) 
    

    if (soup_offer_text.find(attrs={"data-at": "section-text-introduction-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-introduction-content"}).span is not None):
            introduction = str(soup_offer_text.find(attrs={"data-at": "section-text-introduction-content"}).span.contents[0]) 


    if (soup_offer_text.find(attrs={"data-at": "section-text-description-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-description-content"}).span is not None):
            description = str(soup_offer_text.find(attrs={"data-at": "section-text-description-content"}).span.contents[0])


    if (soup_offer_text.find(attrs={"data-at": "section-text-profile-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-profile-content"}).span is not None):
            profile = str(soup_offer_text.find(attrs={"data-at": "section-text-profile-content"}).span.contents[0])


    if (soup_offer_text.find(attrs={"data-at": "section-text-benefits-content"}) is not None):
        if (soup_offer_text.find(attrs={"data-at": "section-text-benefits-content"}).span is not None):
            benefits = str(soup_offer_text.find(attrs={"data-at": "section-text-benefits-content"}).span.contents[0])

    texts = [introduction, description, profile, benefits]
    text = open("rohtext.txt", "a", encoding='utf-16-le')
    text.write("\n" + "## ")
    text.write("///" + title+ "///" + "\n")
    for x in texts:   
        text.write(x + "\n")
    text.close()

text = open("rohtext.txt", "w", encoding='utf-16-le')
text.close()
i = 0

with open("Links_manuell.txt") as read_file:
     file = read_file.read()  
for url in file.split('\n'): 
    offer_url = url 

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "de-DE,de;q=0.5","Accept-Encoding": "gzip, deflate"}
    
    matches = 500 

    sourcecode_offer = requests.get(url, headers=headers).text
    soup_offer = BeautifulSoup(sourcecode_offer, "lxml")

    extract(soup_offer)
    i += 1
    print(str(i) + "/" + str(matches) + "---" + format((i/matches)* 100, '.2f') + "%") 

In [ ]:
#‼️ Schritte tlws. redudant, wenn mit .text-Attribut gearbeitet worden wäre, da inline-Tags dadurch automatisch übersprungen werden
#‼️ Arbeit mit regex aber lobenswert

import re 

#‼️ with-Statement benutzen
text = open("rohtext.txt", "r", encoding='utf-16-le') 
lines = text.readlines()

korpus = "".join(lines)
    
remove = ["<p>", "</p>", "<strong>", "</strong>", "<ul>", "</ul>", "<li>", "</li>", "<em>", "</em>", "<br/>", "</a>","<b>", "</b>", "<u>", "</u>", "<i>", "</i>"]
for x in remove:
    korpus = korpus.replace(x, " ")

korpus = re.sub(r'<a href=".*">', "", korpus)

korpus = re.sub(r'<h1>.*</h1>', "", korpus)

korpus = korpus.replace("\n", " ")

korpus = korpus.replace("   ", " ")
while "  " in korpus:
    korpus = korpus.replace("  ", " ")

korpus = korpus.replace("„", "\"")
korpus = korpus.replace("“", "\"")
korpus = korpus.replace("·", "-")
korpus = korpus.replace("‑","-")
korpus = korpus.replace("’","'")
korpus = korpus.replace("…","...")
korpus = korpus.replace("\t"," ")

korpus = re.sub(r'[^a-zA-Z0-9äöü.,;ß/()\[\]\-#\|&\*\+:ÖÄÜ?"!§$%&\\²³€\'\s]', "", korpus)

text = open("Dachau.txt", "w", encoding='utf-16-le')   
text.write(korpus)
text.close()

In [ ]:
#‼️ für Datenanalyse effizienter: pandas
import re

korpus = open("Dachau.txt", "r", encoding='utf-16-le') 
lines = korpus.readlines()

korpus = "".join(lines)
korpus = korpus.strip()

korpus = korpus.split("##")
print(len(korpus))
job_text = {} 
korpus.remove('') 

title_regex = r'///.*///'
for x in korpus:
    inseration = x.strip()
    title = str(re.findall(title_regex, inseration).pop())

    inseration = inseration.replace(title,"")
    inseration.strip()
    title = title.replace("///","")
    job_text[title] = inseration
num_jobs = len(job_text)


print("sternchen") 
sternchen_dict = {}
sternchen_regex =r'.\*(\s|\))'

for title in job_text:
    if len(re.findall(sternchen_regex, title)) != 0:
        print(title)
        sternchen_dict[title] = job_text[title]

for title in sternchen_dict:
    job_text.pop(title, None)
print("-----") 

print("in-Formen")
in_dict = {} 
in_regex = r'\w*(\*|:|_)\s*in'

for title in job_text:
    if len(re.findall(in_regex, title)) != 0:
        print(title)
        in_dict[title] = job_text[title]

for title in in_dict:
    job_text.pop(title, None)
print("-----")     

print("m/d/w-Formen")
mwd_dict = {} 
mwd_regex = r'(\(|\s)(\w(\/|\||,)\s*\w(\/|\||,)\s*\w)|(\w(\/|\||,)\s*\w)(\)|\s)'

for title in job_text:
    
    if len(re.findall(mwd_regex, title)) != 0:
        print(title)
        mwd_dict[title] = job_text[title]


for title in mwd_dict:
    job_text.pop(title, None)
print("-----")

print("Rest")
for title in job_text:
    print(title)
print("Jobs insg.: " + str(num_jobs))
print("Jobs Sternchen: " + str(len(sternchen_dict)))
print("Jobs *_:in: " + str(len(in_dict)))
print("Jobs m/w/d: " + str(len(mwd_dict)))